## Original

In [2]:
import numpy as np

from coco_mulla.utilities.symbolic_utils import reduce_piano, process_chord, process_midi
from coco_mulla.utilities.sep_utils import separate
from coco_mulla.utilities.encodec_utils import extract_rvq, extract_musicgen_emb
from coco_mulla.utilities import *
import librosa
from config import *

device = get_device()

def process_single(output_folder, path_dict, fname):
    output_folder = os.path.join(output_folder, fname)
    print("begin", output_folder)
    mkdir(output_folder)
    audio_path = path_dict["audio"]
    chord_path = path_dict["chord"]
    midi_path = path_dict["midi"]
    sr = 48000

    drums_output_path = os.path.join(output_folder, "drums_rvq.npy")
    mix_output_path = os.path.join(output_folder, "mix_rvq.npy")
    chord_output_path = os.path.join(output_folder, "chord.npy")
    midi_output_path = os.path.join(output_folder, "midi.npy")
    print(mix_output_path)
    if os.path.exists(mix_output_path):
        print(mix_output_path, "skip")
        return

    flatten_midi_path = midi_path+".piano.mid"
    if not os.path.exists(flatten_midi_path):
        reduce_piano(midi_path, reduced_path=flatten_midi_path)

    wav, _ = librosa.load(audio_path, sr=sr, mono=True)

    wav = np2torch(wav).to(device)[None, None, ...]
    wavs = separate(wav, sr)
    print("separate", output_folder)

    drums_rvq = extract_rvq(wavs["drums"], sr=sr)
    mix_rvq = extract_rvq(wav, sr=sr)

    chord, _ = process_chord(chord_path)
    piano_roll, _ = process_midi(flatten_midi_path)
    print('chord before', chord.shape)
    print('piano before', piano_roll.shape)
    print('drums rvq',drums_rvq.shape)
    print('mix rvq',mix_rvq.shape)
    
    max_len = 11718

    if len(chord) < max_len:
        t_len = len(chord)
        chord = np.pad(chord, ((0, max_len - len(chord)), (0, 0)), "constant", constant_values=(0, 0))
        chord[t_len:, -1] = 1
    else:
        chord = chord[:max_len]
    print(max_len, len(piano_roll), max_len-len(piano_roll))
    piano_roll = np.pad(piano_roll, ((0, max_len - len(piano_roll)), (0, 0)), "constant", constant_values=(0, 0))
    print('chord after', chord.shape)
    print('piano after', piano_roll.shape)
    np.save(chord_output_path, chord)
    np.save(midi_output_path, piano_roll)

    np.save(drums_output_path, drums_rvq.cpu().numpy())
    np.save(mix_output_path, mix_rvq.cpu().numpy())


def scan_audio(audio_folder, low, up):
    res = {}
    for song in os.listdir(audio_folder):
        fname = song
        print(fname)
        audio_path = os.path.join(audio_folder, song, song+".flac")
        print(audio_path)
        midi_path = os.path.join(audio_folder, song, song+".mid")

        # if int(fname) < low or int(fname) >= up:
        #     continue
        res[fname] = {
            "audio": audio_path,
            "midi": midi_path,
            "chord": audio_path + ".chord.lab",
        }
    return res

def process_all(audio_folder, output_folder, low, up):
    data = scan_audio(audio_folder, low, up)
    for song_name in data:
        process_single(output_folder, data[song_name], song_name)


In [3]:
audio_folder = '/l/users/fathinah.izzati/coco-mulla-repo/demo/inp'
output_folder = '/l/users/fathinah.izzati/coco-mulla-repo/demo/out'
process_all(audio_folder, output_folder, 0, 1000)

let_it_be
/l/users/fathinah.izzati/coco-mulla-repo/demo/inp/let_it_be/let_it_be.flac
begin /l/users/fathinah.izzati/coco-mulla-repo/demo/out/let_it_be
/l/users/fathinah.izzati/coco-mulla-repo/demo/out/let_it_be/mix_rvq.npy
separate /l/users/fathinah.izzati/coco-mulla-repo/demo/out/let_it_be
chord before (11719, 36)
piano before (11532, 128)
drums rvq torch.Size([4, 5859])
mix rvq torch.Size([4, 5859])
11718 11532 186
chord after (11718, 36)
piano after (11718, 128)


In [5]:
import numpy as np
mixrvq = np.load('/l/users/fathinah.izzati/coco-mulla-repo/demo/training_input/let_it_be/mix_rvq.npy')
chord = np.load('/l/users/fathinah.izzati/coco-mulla-repo/demo/training_input/let_it_be/chord.npy')
midi = np.load('/l/users/fathinah.izzati/coco-mulla-repo/demo/training_input/let_it_be/midi.npy')
drum = np.load('/l/users/fathinah.izzati/coco-mulla-repo/demo/training_input/let_it_be/drums_rvq.npy')

In [7]:
print(mixrvq.shape)
print(chord.shape)
print(midi.shape)
print(drum.shape)

(4, 11718)
(11718, 36)
(11718, 128)
(4, 11718)


In [ ]:
import numpy as np

from ..utilities.symbolic_utils import reduce_piano, process_chord, process_midi
from ..utilities.sep_utils import separate
from ..utilities.encodec_utils import extract_rvq, extract_musicgen_emb
from ..utilities import *
import librosa
from config import *

device = get_device()


def process_single(output_folder, path_dict, fname):
    output_folder = os.path.join(output_folder, fname)
    print("begin", output_folder)
    mkdir(output_folder)
    audio_path = path_dict["audio"]
    chord_path = path_dict["chord"]
    midi_path = path_dict["midi"]
    sr = 48000

    drums_output_path = os.path.join(output_folder, "drums_rvq.npy")
    mix_output_path = os.path.join(output_folder, "mix_rvq.npy")
    chord_output_path = os.path.join(output_folder, "chord.npy")
    midi_output_path = os.path.join(output_folder, "midi.npy")
    print(mix_output_path)
    if os.path.exists(mix_output_path):
        print(mix_output_path, "skip")
        return

    flatten_midi_path = midi_path+".piano.mid"
    if not os.path.exists(flatten_midi_path):
        reduce_piano(midi_path, reduced_path=flatten_midi_path)

    wav, _ = librosa.load(audio_path, sr=sr, mono=True)

    wav = np2torch(wav).to(device)[None, None, ...]
    wavs = separate(wav, sr)
    print("separate", output_folder)

    drums_rvq = extract_rvq(wavs["drums"], sr=sr)
    mix_rvq = extract_rvq(wav, sr=sr)

    chord, _ = process_chord(chord_path)
    piano_roll, _ = process_midi(flatten_midi_path)
    max_len = len(drums_rvq[0])

    if len(chord) < max_len:
        t_len = len(chord)
        chord = np.pad(chord, ((0, max_len - len(chord)), (0, 0)), "constant", constant_values=(0, 0))
        chord[t_len:, -1] = 1
    else:
        chord = chord[:max_len]
    piano_roll = np.pad(piano_roll, ((0, max_len - len(piano_roll)), (0, 0)), "constant", constant_values=(0, 0))

    np.save(chord_output_path, chord)
    np.save(midi_output_path, piano_roll)




    np.save(drums_output_path, drums_rvq.cpu().numpy())
    np.save(mix_output_path, mix_rvq.cpu().numpy())


def scan_audio(audio_folder):
    res = {}
    for song in os.listdir(audio_folder):
        fname = song
        audio_path = os.path.join(audio_folder, song, "mix.flac")
        midi_path = os.path.join(audio_folder, song, "all_src.mid")

        res[fname] = {
            "audio": audio_path,
            "midi": midi_path,
            "chord": audio_path + ".chord.lab",
        }
    return res


def process_all(audio_folder, output_folder):
    data = scan_audio(audio_folder, low, up)
    for song_name in data:
        process_single(output_folder, data[song_name], song_name)